## Homework

> Note: sometimes your answer doesn't match one of 
> the options exactly. That's fine. 
> Select the option that's closest to your solution.
> If it's exactly in between two options, select the higher value.


### Dataset

In this homework, we continue using the fuel efficiency dataset.
Download it from <a href='https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv'>here</a>.

You can do it with wget:

```bash
wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv
```

The goal of this homework is to create a regression model for predicting the car fuel efficiency (column `'fuel_efficiency_mpg'`).



### Preparing the dataset 

Preparation:

* Fill missing values with zeros.
* Do train/validation/test split with 60%/20%/20% distribution. 
* Use the `train_test_split` function and set the `random_state` parameter to 1.
* Use `DictVectorizer(sparse=True)` to turn the dataframes into matrices.



In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [12]:
# Download and load the dataset

In [3]:
url = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv'
df = pd.read_csv(url)

In [4]:
print("Dataset shape:", df.shape)
print("\nFirst few rows:")
print(df.head())
print("\nColumn names:")
print(df.columns.tolist())
print("\nMissing values:")
print(df.isnull().sum())

Dataset shape: (9704, 11)

First few rows:
   engine_displacement  num_cylinders  horsepower  vehicle_weight  \
0                  170            3.0       159.0     3413.433759   
1                  130            5.0        97.0     3149.664934   
2                  170            NaN        78.0     3079.038997   
3                  220            4.0         NaN     2542.392402   
4                  210            1.0       140.0     3460.870990   

   acceleration  model_year  origin fuel_type         drivetrain  num_doors  \
0          17.7        2003  Europe  Gasoline    All-wheel drive        0.0   
1          17.8        2007     USA  Gasoline  Front-wheel drive        0.0   
2          15.1        2018  Europe  Gasoline  Front-wheel drive        0.0   
3          20.2        2009     USA    Diesel    All-wheel drive        2.0   
4          14.4        2009  Europe  Gasoline    All-wheel drive        2.0   

   fuel_efficiency_mpg  
0            13.231729  
1            13.6

In [6]:
# Prepare the dataset
# Fill missing values with zeros
df = df.fillna(0)

In [16]:
# Split into features and target
y = df['fuel_efficiency_mpg'].values
X = df.drop('fuel_efficiency_mpg', axis=1)

In [17]:
# Train/validation/test split (60%/20%/20%)
X_full_train, X_test, y_full_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1)

In [18]:
X_train, X_val, y_train, y_val = train_test_split(
    X_full_train, y_full_train, test_size=0.25, random_state=1  # 0.25 * 0.8 = 0.2
)

In [19]:
print(f"Train size: {len(X_train)}")
print(f"\nValidation size: {len(X_val)}")
print(f"\nTest size: {len(X_test)}")

Train size: 5822

Validation size: 1941

Test size: 1941


In [20]:
# Convert to DictVectorizer
dv = DictVectorizer(sparse=True)

In [21]:
train_dicts = X_train.to_dict(orient='records')
X_train_sparse = dv.fit_transform(train_dicts)

In [22]:
val_dicts = X_val.to_dict(orient='records')
X_val_sparse = dv.transform(val_dicts)


In [23]:
test_dicts = X_test.to_dict(orient='records')
X_test_sparse = dv.transform(test_dicts)

In [24]:
print(f"Feature names: {dv.get_feature_names_out()}")


Feature names: ['acceleration' 'drivetrain=All-wheel drive'
 'drivetrain=Front-wheel drive' 'engine_displacement' 'fuel_type=Diesel'
 'fuel_type=Gasoline' 'horsepower' 'model_year' 'num_cylinders'
 'num_doors' 'origin=Asia' 'origin=Europe' 'origin=USA' 'vehicle_weight']


## Question 1

Let's train a decision tree regressor to predict the `fuel_efficiency_mpg` variable. 

* Train a model with `max_depth=1`.


Which feature is used for splitting the data?


* `'vehicle_weight'`
* `'model_year'`
* `'origin'`
* `'fuel_type'`


In [25]:
# Question 1: Decision Tree with max_depth=1
print("="*50)
print("QUESTION 1: Decision Tree with max_depth=1")
print("="*50)

QUESTION 1: Decision Tree with max_depth=1


In [26]:

dt = DecisionTreeRegressor(max_depth=1, random_state=1)
dt.fit(X_train_sparse, y_train)


DecisionTreeRegressor(max_depth=1, random_state=1)

In [27]:
# Get the feature used for splitting
feature_idx = dt.tree_.feature[0]
feature_name = dv.get_feature_names_out()[feature_idx]
print(f"Feature used for splitting: {feature_name}")


Feature used for splitting: vehicle_weight


## Question 2

Train a random forest regressor with these parameters:

* `n_estimators=10`
* `random_state=1`
* `n_jobs=-1` (optional - to make training faster)


What's the RMSE of this model on the validation data?

* 0.045
* 0.45
* 4.5
* 45.0

In [28]:
# Question 2: Random Forest with n_estimators=10
print("="*50)
print("QUESTION 2: Random Forest RMSE")
print("="*50)

QUESTION 2: Random Forest RMSE


In [29]:
rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train_sparse, y_train)

RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=1)

In [30]:
y_val_pred = rf.predict(X_val_sparse)
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f"RMSE on validation data: {rmse:.3f}")


RMSE on validation data: 0.460


## Question 3

Now let's experiment with the `n_estimators` parameter

* Try different values of this parameter from 10 to 200 with step 10.
* Set `random_state` to `1`.
* Evaluate the model on the validation dataset.


After which value of `n_estimators` does RMSE stop improving?
Consider 3 decimal places for calculating the answer.

- 10
- 25
- 80
- 200

If it doesn't stop improving, use the latest iteration number in
your answer.

In [31]:
# Question 3: Experiment with n_estimators
print("="*50)
print("QUESTION 3: n_estimators experiments")
print("="*50)

QUESTION 3: n_estimators experiments


In [32]:
n_estimators_values = range(10, 201, 10)
rmse_scores = []

for n in n_estimators_values:
    rf = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rf.fit(X_train_sparse, y_train)
    y_val_pred = rf.predict(X_val_sparse)
    rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    rmse_scores.append(rmse)
    print(f"n_estimators={n:3d}, RMSE={rmse:.3f}")
    

n_estimators= 10, RMSE=0.460
n_estimators= 20, RMSE=0.454
n_estimators= 30, RMSE=0.452
n_estimators= 40, RMSE=0.449
n_estimators= 50, RMSE=0.447
n_estimators= 60, RMSE=0.445
n_estimators= 70, RMSE=0.445
n_estimators= 80, RMSE=0.445
n_estimators= 90, RMSE=0.445
n_estimators=100, RMSE=0.445
n_estimators=110, RMSE=0.444
n_estimators=120, RMSE=0.444
n_estimators=130, RMSE=0.444
n_estimators=140, RMSE=0.443
n_estimators=150, RMSE=0.443
n_estimators=160, RMSE=0.443
n_estimators=170, RMSE=0.443
n_estimators=180, RMSE=0.442
n_estimators=190, RMSE=0.442
n_estimators=200, RMSE=0.442


In [33]:
# Find when RMSE stops improving (3 decimal places)
min_rmse = min(rmse_scores)
min_idx = rmse_scores.index(min_rmse)
best_n_estimators = list(n_estimators_values)[min_idx]
print(f"Best n_estimators: {best_n_estimators} with RMSE: {min_rmse:.3f}")

Best n_estimators: 180 with RMSE: 0.442


In [34]:
# Check when it stops improving
for i in range(len(rmse_scores) - 1):
    if round(rmse_scores[i], 3) == round(min_rmse, 3):
        print(f"RMSE stops improving at n_estimators={list(n_estimators_values)[i]}")
        break

RMSE stops improving at n_estimators=180


## Question 4

Let's select the best `max_depth`:

* Try different values of `max_depth`: `[10, 15, 20, 25]`
* For each of these values,
  * try different values of `n_estimators` from 10 till 200 (with step 10)
  * calculate the mean RMSE 
* Fix the random seed: `random_state=1`


What's the best `max_depth`, using the mean RMSE?

* 10
* 15
* 20
* 25

In [35]:
# Question 4: Select best max_depth
print("="*50)
print("QUESTION 4: Best max_depth")
print("="*50)

QUESTION 4: Best max_depth


In [36]:
max_depth_values = [10, 15, 20, 25]
results = {}

for max_depth in max_depth_values:
    rmse_list = []
    for n in range(10, 201, 10):
        rf = RandomForestRegressor(
            n_estimators=n, 
            max_depth=max_depth, 
            random_state=1, 
            n_jobs=-1
        )
        rf.fit(X_train_sparse, y_train)
        y_val_pred = rf.predict(X_val_sparse)
        rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
        rmse_list.append(rmse)
    
    mean_rmse = np.mean(rmse_list)
    results[max_depth] = mean_rmse
    print(f"max_depth={max_depth}, mean RMSE={mean_rmse:.3f}")

max_depth=10, mean RMSE=0.442
max_depth=15, mean RMSE=0.445
max_depth=20, mean RMSE=0.446
max_depth=25, mean RMSE=0.446


In [37]:
best_max_depth = min(results, key=results.get)
print(f"Best max_depth: {best_max_depth}")

Best max_depth: 10


# Question 5

We can extract feature importance information from tree-based models. 

At each step of the decision tree learning algorithm, it finds the best split. 
When doing it, we can calculate "gain" - the reduction in impurity before and after the split. 
This gain is quite useful in understanding what are the important features for tree-based models.

In Scikit-Learn, tree-based models contain this information in the
[`feature_importances_`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor.feature_importances_)
field. 

For this homework question, we'll find the most important feature:

* Train the model with these parameters:
  * `n_estimators=10`,
  * `max_depth=20`,
  * `random_state=1`,
  * `n_jobs=-1` (optional)
* Get the feature importance information from this model


What's the most important feature (among these 4)? 

* `vehicle_weight`
*	`horsepower`
* `acceleration`
* `engine_displacement`	

In [38]:
# Question 5: Feature importance
print("="*50)
print("QUESTION 5: Feature importance")
print("="*50)

QUESTION 5: Feature importance


In [39]:
rf = RandomForestRegressor(
    n_estimators=10, 
    max_depth=20, 
    random_state=1, 
    n_jobs=-1
)

In [40]:
rf.fit(X_train_sparse, y_train)

RandomForestRegressor(max_depth=20, n_estimators=10, n_jobs=-1, random_state=1)

In [41]:
feature_names = dv.get_feature_names_out()
feature_importance = rf.feature_importances_

In [42]:
# Create a dictionary of feature importance
importance_dict = dict(zip(feature_names, feature_importance))

In [43]:
# Look for the specific features
important_features = ['vehicle_weight', 'horsepower', 'acceleration', 'engine_displacement']
print("\nFeature importances for specified features:")
for feature in important_features:
    if feature in importance_dict:
        print(f"{feature}: {importance_dict[feature]:.6f}")


Feature importances for specified features:
vehicle_weight: 0.959150
horsepower: 0.015998
acceleration: 0.011480
engine_displacement: 0.003273


In [44]:
# Find most important overall
sorted_importance = sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)
print(f"Top 10 most important features:")
for feature, importance in sorted_importance[:10]:
    print(f"{feature}: {importance:.6f}")

Top 10 most important features:
vehicle_weight: 0.959150
horsepower: 0.015998
acceleration: 0.011480
engine_displacement: 0.003273
model_year: 0.003212
num_cylinders: 0.002343
num_doors: 0.001635
origin=USA: 0.000540
origin=Europe: 0.000519
origin=Asia: 0.000462


## Question 6

Now let's train an XGBoost model! For this question, we'll tune the `eta` parameter:

* Install XGBoost
* Create DMatrix for train and validation
* Create a watchlist
* Train a model with these parameters for 100 rounds:

```
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
```

Now change `eta` from `0.3` to `0.1`.

Which eta leads to the best RMSE score on the validation dataset?

* 0.3
* 0.1
* Both give equal value

In [45]:
# Question 6: XGBoost with different eta values
print("="*50)
print("QUESTION 6: XGBoost eta tuning")
print("="*50)

QUESTION 6: XGBoost eta tuning


In [46]:
# Create DMatrix
dtrain = xgb.DMatrix(X_train_sparse, label=y_train)
dval = xgb.DMatrix(X_val_sparse, label=y_val)

In [47]:
watchlist = [(dtrain, 'train'), (dval, 'val')]

In [48]:
# Test eta=0.3
xgb_params_03 = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

In [49]:
print("Training with eta=0.3:")
model_03 = xgb.train(xgb_params_03, dtrain, num_boost_round=100, 
                      evals=watchlist, verbose_eval=20)

Training with eta=0.3:
[0]	train-rmse:1.81393	val-rmse:1.85444
[20]	train-rmse:0.33553	val-rmse:0.43376
[40]	train-rmse:0.30202	val-rmse:0.43968
[60]	train-rmse:0.26768	val-rmse:0.44290
[80]	train-rmse:0.24254	val-rmse:0.44689
[99]	train-rmse:0.21950	val-rmse:0.45018


In [50]:
y_val_pred_03 = model_03.predict(dval)
rmse_03 = np.sqrt(mean_squared_error(y_val, y_val_pred_03))
print(f"RMSE with eta=0.3: {rmse_03:.6f}")

RMSE with eta=0.3: 0.450178


In [51]:
# Test eta=0.1
xgb_params_01 = {
    'eta': 0.1, 
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

In [52]:
print("Training with eta=0.1:")
model_01 = xgb.train(xgb_params_01, dtrain, num_boost_round=100, 
                      evals=watchlist, verbose_eval=20)

Training with eta=0.1:
[0]	train-rmse:2.28944	val-rmse:2.34561
[20]	train-rmse:0.48983	val-rmse:0.53064
[40]	train-rmse:0.35343	val-rmse:0.42746
[60]	train-rmse:0.33054	val-rmse:0.42456
[80]	train-rmse:0.31667	val-rmse:0.42563
[99]	train-rmse:0.30419	val-rmse:0.42623


In [53]:
y_val_pred_01 = model_01.predict(dval)
rmse_01 = np.sqrt(mean_squared_error(y_val, y_val_pred_01))
print(f"RMSE with eta=0.1: {rmse_01:.6f}")

RMSE with eta=0.1: 0.426228
